## Xavier initialization
### global이 아닌 local minima를 찾는 문제가 있음 => initialization을 통해 해결
### relu 함수에 최적화된 HE initialization => 분산=4/Channel_in + Channel_out

# Xavier Deep network version

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


# Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        
        weight_init = tf.keras.initializers.glorot_uniform() ## Initializator

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.9258, train_loss: 2.02812147, train_accuracy: 0.6094, test_Accuracy: 0.3990
Epoch: [ 0] [    1/  468] time: 1.3499, train_loss: 1.89642477, train_accuracy: 0.5938, test_Accuracy: 0.4280
Epoch: [ 0] [    2/  468] time: 1.8846, train_loss: 1.66584527, train_accuracy: 0.7031, test_Accuracy: 0.6344
Epoch: [ 0] [    3/  468] time: 2.3100, train_loss: 1.51968563, train_accuracy: 0.7500, test_Accuracy: 0.7219
Epoch: [ 0] [    4/  468] time: 2.6941, train_loss: 1.23590767, train_accuracy: 0.7188, test_Accuracy: 0.6958
Epoch: [ 0] [    5/  468] time: 3.0764, train_loss: 0.91388392, train_accuracy: 0.7266, test_Accuracy: 0.6855
Epoch: [ 0] [    6/  468] time: 3.6876, train_loss: 0.83714324, train_accuracy: 0.8125, test_Accuracy: 0.7773
Epoch: [ 0] [    7/  468] time: 4.0460, train_loss: 0.72497058, train_accuracy: 0.8359, test_Accuracy: 0.7597
Epoch: [ 0] [    8/  468] time: 4.6698,

Epoch: [ 0] [   74/  468] time: 31.9933, train_loss: 0.25734606, train_accuracy: 0.9453, test_Accuracy: 0.9331
Epoch: [ 0] [   75/  468] time: 32.3259, train_loss: 0.23960394, train_accuracy: 0.9141, test_Accuracy: 0.9285
Epoch: [ 0] [   76/  468] time: 32.7200, train_loss: 0.19021530, train_accuracy: 0.9219, test_Accuracy: 0.9117
Epoch: [ 0] [   77/  468] time: 33.0492, train_loss: 0.22819486, train_accuracy: 0.9375, test_Accuracy: 0.9113
Epoch: [ 0] [   78/  468] time: 33.4821, train_loss: 0.15290941, train_accuracy: 0.9609, test_Accuracy: 0.9223
Epoch: [ 0] [   79/  468] time: 33.8945, train_loss: 0.28150165, train_accuracy: 0.9219, test_Accuracy: 0.9350
Epoch: [ 0] [   80/  468] time: 34.3008, train_loss: 0.21952650, train_accuracy: 0.9375, test_Accuracy: 0.9275
Epoch: [ 0] [   81/  468] time: 34.7439, train_loss: 0.31461194, train_accuracy: 0.9141, test_Accuracy: 0.9185
Epoch: [ 0] [   82/  468] time: 35.2436, train_loss: 0.15799782, train_accuracy: 0.9375, test_Accuracy: 0.9160
E

Epoch: [ 0] [  148/  468] time: 62.1719, train_loss: 0.15495758, train_accuracy: 0.9453, test_Accuracy: 0.9433
Epoch: [ 0] [  149/  468] time: 62.5161, train_loss: 0.09682118, train_accuracy: 0.9688, test_Accuracy: 0.9347
Epoch: [ 0] [  150/  468] time: 62.8571, train_loss: 0.17017466, train_accuracy: 0.9531, test_Accuracy: 0.9366
Epoch: [ 0] [  151/  468] time: 63.2884, train_loss: 0.18391971, train_accuracy: 0.9531, test_Accuracy: 0.9391
Epoch: [ 0] [  152/  468] time: 63.6605, train_loss: 0.20483783, train_accuracy: 0.9531, test_Accuracy: 0.9418
Epoch: [ 0] [  153/  468] time: 64.1583, train_loss: 0.20867747, train_accuracy: 0.9453, test_Accuracy: 0.9470
Epoch: [ 0] [  154/  468] time: 64.5618, train_loss: 0.11759378, train_accuracy: 0.9609, test_Accuracy: 0.9452
Epoch: [ 0] [  155/  468] time: 64.9894, train_loss: 0.26854244, train_accuracy: 0.9141, test_Accuracy: 0.9411
Epoch: [ 0] [  156/  468] time: 65.5332, train_loss: 0.10505683, train_accuracy: 0.9609, test_Accuracy: 0.9376
E

Epoch: [ 0] [  222/  468] time: 92.1666, train_loss: 0.06751882, train_accuracy: 0.9844, test_Accuracy: 0.9560
Epoch: [ 0] [  223/  468] time: 92.6395, train_loss: 0.09426358, train_accuracy: 0.9688, test_Accuracy: 0.9545
Epoch: [ 0] [  224/  468] time: 93.0761, train_loss: 0.08491783, train_accuracy: 0.9688, test_Accuracy: 0.9529
Epoch: [ 0] [  225/  468] time: 93.4583, train_loss: 0.13329831, train_accuracy: 0.9766, test_Accuracy: 0.9507
Epoch: [ 0] [  226/  468] time: 93.7867, train_loss: 0.14510013, train_accuracy: 0.9375, test_Accuracy: 0.9534
Epoch: [ 0] [  227/  468] time: 94.1261, train_loss: 0.10130331, train_accuracy: 0.9766, test_Accuracy: 0.9563
Epoch: [ 0] [  228/  468] time: 94.4739, train_loss: 0.05185942, train_accuracy: 0.9922, test_Accuracy: 0.9565
Epoch: [ 0] [  229/  468] time: 94.8012, train_loss: 0.12026276, train_accuracy: 0.9609, test_Accuracy: 0.9582
Epoch: [ 0] [  230/  468] time: 95.2094, train_loss: 0.08360844, train_accuracy: 0.9766, test_Accuracy: 0.9589
E

Epoch: [ 0] [  296/  468] time: 124.3089, train_loss: 0.14005783, train_accuracy: 0.9531, test_Accuracy: 0.9638
Epoch: [ 0] [  297/  468] time: 124.7685, train_loss: 0.17078030, train_accuracy: 0.9531, test_Accuracy: 0.9626
Epoch: [ 0] [  298/  468] time: 125.1685, train_loss: 0.07599905, train_accuracy: 0.9766, test_Accuracy: 0.9634
Epoch: [ 0] [  299/  468] time: 125.5960, train_loss: 0.06944077, train_accuracy: 0.9844, test_Accuracy: 0.9625
Epoch: [ 0] [  300/  468] time: 126.0213, train_loss: 0.04944002, train_accuracy: 0.9922, test_Accuracy: 0.9614
Epoch: [ 0] [  301/  468] time: 126.4350, train_loss: 0.17815892, train_accuracy: 0.9609, test_Accuracy: 0.9617
Epoch: [ 0] [  302/  468] time: 126.7754, train_loss: 0.08685820, train_accuracy: 0.9844, test_Accuracy: 0.9621
Epoch: [ 0] [  303/  468] time: 127.1730, train_loss: 0.13351323, train_accuracy: 0.9531, test_Accuracy: 0.9622
Epoch: [ 0] [  304/  468] time: 127.5442, train_loss: 0.15919581, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 154.7909, train_loss: 0.04964381, train_accuracy: 0.9844, test_Accuracy: 0.9610
Epoch: [ 0] [  371/  468] time: 155.2402, train_loss: 0.12169806, train_accuracy: 0.9531, test_Accuracy: 0.9643
Epoch: [ 0] [  372/  468] time: 155.6352, train_loss: 0.11334420, train_accuracy: 0.9688, test_Accuracy: 0.9648
Epoch: [ 0] [  373/  468] time: 156.0639, train_loss: 0.13712569, train_accuracy: 0.9609, test_Accuracy: 0.9645
Epoch: [ 0] [  374/  468] time: 156.4895, train_loss: 0.16996419, train_accuracy: 0.9453, test_Accuracy: 0.9637
Epoch: [ 0] [  375/  468] time: 156.9403, train_loss: 0.12139468, train_accuracy: 0.9688, test_Accuracy: 0.9632
Epoch: [ 0] [  376/  468] time: 157.4251, train_loss: 0.08404960, train_accuracy: 0.9844, test_Accuracy: 0.9636
Epoch: [ 0] [  377/  468] time: 157.8800, train_loss: 0.11493848, train_accuracy: 0.9688, test_Accuracy: 0.9628
Epoch: [ 0] [  378/  468] time: 158.2765, train_loss: 0.16986760, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 185.4022, train_loss: 0.05185466, train_accuracy: 0.9766, test_Accuracy: 0.9708
Epoch: [ 0] [  445/  468] time: 185.7486, train_loss: 0.06486334, train_accuracy: 0.9766, test_Accuracy: 0.9701
Epoch: [ 0] [  446/  468] time: 186.0980, train_loss: 0.03737441, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [  447/  468] time: 186.4601, train_loss: 0.08123229, train_accuracy: 0.9688, test_Accuracy: 0.9675
Epoch: [ 0] [  448/  468] time: 186.8806, train_loss: 0.09248833, train_accuracy: 0.9766, test_Accuracy: 0.9664
Epoch: [ 0] [  449/  468] time: 187.3616, train_loss: 0.11323323, train_accuracy: 0.9766, test_Accuracy: 0.9685
Epoch: [ 0] [  450/  468] time: 187.7185, train_loss: 0.16114613, train_accuracy: 0.9531, test_Accuracy: 0.9692
Epoch: [ 0] [  451/  468] time: 188.1005, train_loss: 0.02924548, train_accuracy: 0.9922, test_Accuracy: 0.9689
Epoch: [ 0] [  452/  468] time: 188.5595, train_loss: 0.07950802, train_accuracy: 0.9766, test_Accuracy:

## Accuracy : 96.31%
### 함수를 바꾸는것보다 더 확실히 정확도가 향상되었음.